### Metadata in Short-Term Memory

Event metadata lets you attach additional context information to your short-term memory events as key-value pairs. When creating events using the CreateEvent operation, you can include metadata that isn't part of the core event content but provides valuable context for retrieval. For example, a travel booking agent can attach location metadata to events, making it easy to find all conversations that mentioned specific destinations. You can then use the ListEvents operation with metadata filters to efficiently retrieve events based on these attached properties, enabling your agent to quickly locate relevant conversation history without scanning through entire sessions. This capability is useful for agents that need to track and retrieve specific attributes across conversations, such as product categories in e-commerce, case types in customer support, or project identifiers in task management applications. Event metadata is not meant to store sensitive content, as it is not encrypted with customer managed key.

Below is a short workflow on how metadata can be attached when creating events along with filtering the conversational history to retrieve relevant memories based on varying conditions

In [ ]:
import time
from typing import Optional

from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager

from bedrock_agentcore.memory import MemorySessionManager
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.models import (
    EventMetadataFilter,
    LeftExpression,
    OperatorType,
    RightExpression,
    StringValue,
)

#### Setting up Memory Resources

In [ ]:
region = "us-west-2"

In [ ]:
memory_manager = MemoryManager(region_name=region)

memory = memory_manager.get_or_create_memory(name="travel_support_agent_1")

In [ ]:
session_manager = MemorySessionManager(memory_id=memory["id"], region_name=region)

session = session_manager.create_memory_session(actor_id="user-123", session_id="session-1")

In [ ]:
event_1 = [
    ConversationalMessage(
        "I am planning to travel to the US next Summer, can you help me plan my trip!", MessageRole.USER
    ),
    ConversationalMessage(
        "That's great to hear! I'd be happy to help you plan the trip. What would be the first city you'd like to visit in the US?",
        MessageRole.ASSISTANT,
    ),
    ConversationalMessage(
        "I am planning on starting off my summer vacation in NYC! I will be visting for 5 days!", MessageRole.USER
    ),
]

metadata_1 = {"location": StringValue.build("NYC"), "season": StringValue.build("Summer")}
session.add_turns(event_1, metadata=metadata_1)
time.sleep(2)  # To avoid being throttled

event_2 = [
    ConversationalMessage(
        "For outdoor experiences you can consider visiting the Central Park, Brooklyn Bridge Park, The High Line.",
        MessageRole.ASSISTANT,
    ),
    ConversationalMessage(
        "That's great to hear, what are some of the classic summer activities I can do?", MessageRole.USER
    ),
    ConversationalMessage(
        "For classic summer activities, you can try visiting: Coney Island, Yankees Game, Statue of Liberty!",
        MessageRole.ASSISTANT,
    ),
    ConversationalMessage("Thank you for helping me in providing these suggestions", MessageRole.USER),
]

metadata_2 = {
    "location": StringValue.build("NYC"),
    "season": StringValue.build("Summer"),
    "attractions": StringValue.build("Central Park/Brooklyn Bridge Park/High Line"),
    "activities": StringValue.build("Coney Island/Yankees Game/Statue of Liberty"),
}
session.add_turns(event_2, metadata=metadata_2)
time.sleep(2)  # To avoid being throttled

event_3 = [
    ConversationalMessage("After NYC, where would you like to visit next!?", MessageRole.ASSISTANT),
    ConversationalMessage("I would be visiting Chicago next!", MessageRole.USER),
    ConversationalMessage(
        "Would you like me to provide you suggestion on how to spend time in Chicago?", MessageRole.ASSISTANT
    ),
    ConversationalMessage("Yes! However, I would be visiting in Chicago for just 2 days!", MessageRole.USER),
]

metadata_3 = {"location": StringValue.build("Chicago"), "season": StringValue.build("Summer")}
session.add_turns(event_3, metadata=metadata_3)
time.sleep(2)  # To avoid being throttled

event_4 = [
    ConversationalMessage(
        "Great! Since your visit is short, you can visting the Millennium Park, Skydeck, Chicago Riverwalk!",
        MessageRole.ASSISTANT,
    ),
    ConversationalMessage("Thank you for the suggestion!", MessageRole.USER),
]

metadata_4 = {
    "location": StringValue.build("Chicago"),
    "season": StringValue.build("Summer"),
    "attractions": StringValue.build("Millennium Park/Skydeck/Chicago Riverwalk"),
}
session.add_turns(event_4, metadata=metadata_4)
time.sleep(2)  # To avoid being throttled

In [ ]:
events = session.list_events()
for index, event in enumerate(events, start=1):
    print(f"=== Event #{index} ===")
    print(event)

#### Listing Events with Metadata Filter

In [ ]:
def build_metadata_filter(key: str, operator: OperatorType, val: Optional[str] = None) -> EventMetadataFilter:
    params = {"left_operand": LeftExpression.build(key=key), "operator": operator}
    if val:
        params["right_operand"] = RightExpression.build(value=val)
    return EventMetadataFilter.build_expression(**params)

##### Listing events based on a key-value pairs

In [ ]:
# Example: location = "NYC"

metadata_filter_1 = build_metadata_filter(key="location", operator=OperatorType.EQUALS_TO, val="NYC")

filtered_events_1 = session.list_events(eventMetadata=[metadata_filter_1])

print("=== Listing events with metadata filter, where: key = value ===")
for index, event in enumerate(filtered_events_1, start=1):
    print(f"=== Event #{index} ===")
    print(event)

##### Listing events based on key existence

In [ ]:
# Example: exists(key) = "attractions"

metadata_filter_2 = build_metadata_filter(key="attractions", operator=OperatorType.EXISTS)

filtered_events_2 = session.list_events(eventMetadata=[metadata_filter_2])

print("=== Listing events with metadata filter, where: exists(key) ===")
for index, event in enumerate(filtered_events_2, start=1):
    print(f"=== Event #{index} ===")
    print(event)

##### Listing events based on key non-existence

In [ ]:
# Example: does_not_exist(key) = "activites"
# Note: In the above 4 events created, only 1 event consists of the key "activites" present in its metadata.
# The below listEvents query should return the remaining events.

metadata_filter_3 = build_metadata_filter(key="activities", operator=OperatorType.NOT_EXISTS)

filtered_events_3 = session.list_events(eventMetadata=[metadata_filter_3])

print("=== Listing events with metadata filter, where: does_not_exist(key) ===")
for index, event in enumerate(filtered_events_3, start=1):
    print(f"=== Event #{index} ===")
    print(event)

#### Listing Events with branch and metadata filters

In [ ]:
# Let's branch off of Event #2
root_event = events[2]

branched_event = [
    ConversationalMessage("After NYC, where would you like to visit next!?", MessageRole.ASSISTANT),
    ConversationalMessage(
        "Actually, I changed my mind. I will be visiting NYC during next winter. Could you provide me suggestions on places to visit here?",
        MessageRole.USER,
    ),
    ConversationalMessage(
        "I would be glad to help you! You can visit the iconic Rockefeller Center that has christmas decorations and trees, and also go ice-skating in the Bryant Park",
        MessageRole.ASSISTANT,
    ),
    ConversationalMessage("Thank you for the suggestion", MessageRole.USER),
]

branched_event_metadata = {"location": StringValue.build("NYC"), "season": StringValue.build("Winter")}
branch_name = "branch-1"
branch = {"rootEventId": root_event["eventId"], "name": branch_name}

session.add_turns(branched_event, branch=branch, metadata=branched_event_metadata)

In [ ]:
branch_name = "branch-1"
# List all the events in "branch-1"
filtered_events_4 = session.list_events(branch_name=branch_name, include_parent_branches=True)

print(f"=== Listing events in branch: {branch_name} ===")
for index, event in enumerate(filtered_events_4, start=1):
    print(f"=== Event #{index} ===")
    print(event)

##### Listing events with multiple metadata filters

The ListEvents API accepts a list of metadata filters. 

When there exists more than one metadata filter, an implicit `AND` operation is performed on the metadata filters provided.  
This implies only the retrieval of events that meet the conditions of all the metadata filters that are provided.  

Below is an example of metadata filtering with more than one metadata filter + branch filtering

In [ ]:
# Example:
# Let us consider two metadata filters to be provided when listing events.
# key1 = "location", value1= "NYC"
# key2 = "Season", value1= "Winter"

metadata_filter_4 = build_metadata_filter(key="location", operator=OperatorType.EQUALS_TO, val="NYC")

metadata_filter_5 = build_metadata_filter(key="season", operator=OperatorType.EQUALS_TO, val="Winter")

filtered_events_5 = session.list_events(
    branch_name=branch_name, include_parent_branches=True, eventMetadata=[metadata_filter_4, metadata_filter_5]
)

print("=== Listing events with branch and metadata filters ===")
for index, event in enumerate(filtered_events_5, start=1):
    print(f"=== Event #{index} ===")
    print(event)